## Run default configurations, but with bespoke pwv

Print out Psats, NET, NET_wafer


In [21]:
# See what yaml files are locally available.
!ls *.yaml

CHLAT_20210714.yaml      CHSAT_LFMF_20210714.yaml SAT_LFMF_20210714.yaml
CHSAT_HF_20210714.yaml   SAT_HF_20210714.yaml     SPLAT_20210714.yaml


In [2]:
# Standard imports
import numpy as np
import yaml
import toml
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]

# bolo-calc import
from bolo import Top



## Run
Read yaml files, run bolo_calc, store some outputs into dictionary called "data".
Print some outputs to table.

In [20]:
data = {}

yamlfiles = ['CHLAT_20210714.yaml','SPLAT_20210714.yaml'] # Lats only because we can use the same ,'SAT_LFMF_20210714.yaml','SAT_HF_20210714.yaml']

#read fixed PSATs from .toml file
psat_dict = toml.load("FixedPsats.toml")

pwvs = {}
pwvs['Pole']=[321.0, 448.0]
pwvs['Atacama']=[993.0, 1871.0]

for yamlfile in yamlfiles:
    print(yamlfile)
    dd = yaml.safe_load(open(yamlfile))
    
    # Change elevation to 50 degrees
    dd['instrument']['elevation']=50.0

    version_date = str(dd['version']['date'])
    version_telescope = str(dd['version']['name'])
    print('{0:8s}  Popt(pW) Psat(pW)   NET    NET_wafer'.format(version_telescope))
    del dd['version']

    # location of atmosphere file
    dd['sim_config']['config_dir'] = '../../bolo-calc/config'
    
    ch_names = list(dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'].keys())
    
    #---------
    #convert to Watts
    for channel in ch_names:
        psat_dict[yamlfile.partition('.')[0]][channel] *= 1e-12

    #set psat factor to None & delete psat entry from channel default
    dd['instrument']['channel_default']['psat_factor'] = None
    del dd['instrument']['channel_default']['psat']

    #assign SI psat values at the channel level
    for channel in ch_names:
        dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'][channel]['psat'] = psat_dict[yamlfile.partition('.')[0]][channel]
    #---------
    
    #site = dd['instrument']['site']

    for site in ['Atacama','Pole']:
        dd['instrument']['site']=site
        for pwv in pwvs[site]:
            dd['instrument']['pwv']=pwv
            print('Site=',dd['instrument']['site'])
            print('Elevation = ',dd['instrument']['elevation'])
            print('pwv=',dd['instrument']['pwv'])
            top = Top(**dd)
            top.run()
        
            for chan in top.instrument.tables.keys():
                if chan.split('_')[0] == 'cam' :
                    chname = chan.split('_')[2] + '_' + chan.split('_')[3]
                    #
                    # Grab things we're going to print and give them short names and store them in the dictionary.
                    popt = top.instrument.tables[chan]['opt_power'][0]
                    psat = top.instrument.tables[chan]['P_sat'][0]
                    net = top.instrument.tables[chan]['NET'][0]
                    netwafer = top.instrument.tables[chan]['NET_arr'][0]
                    #
                    print('{0:5s} : {1:8.2f} {2:8.2f} {3:8.2f} {4:8.2f}'.format(chname,popt,psat,net,netwafer) )
            print('')
        print('--------------------')
    

CHLAT_20210714.yaml
CHLAT     Popt(pW) Psat(pW)   NET    NET_wafer
Site= Atacama
Elevation =  50.0
pwv= 993.0
LF_1  :     0.26     0.75   307.74    35.12
LF_2  :     1.28     4.18   240.72    27.47
MF_1  :     1.45     4.60   278.08    10.58
MF_2  :     4.40    12.51   309.65    11.78
HF_1  :    11.43    31.70   683.16    24.94
HF_2  :    15.38    42.43  1679.70    61.32

Site= Atacama
Elevation =  50.0
pwv= 1871.0
LF_1  :     0.28     0.75   318.43    36.34
LF_2  :     1.30     4.18   243.54    27.79
MF_1  :     1.56     4.60   291.22    11.08
MF_2  :     5.40    12.51   365.42    13.90
HF_1  :    15.30    31.70   890.42    32.50
HF_2  :    21.46    42.43  2345.75    85.63

--------------------
Site= Pole
Elevation =  50.0
pwv= 321.0
LF_1  :     0.27     0.75   314.33    35.87
LF_2  :     1.58     4.18   291.84    33.30
MF_1  :     1.70     4.60   311.24    11.84
MF_2  :     4.20    12.51   298.56    11.36
HF_1  :     9.73    31.70   601.06    21.94
HF_2  :    12.39    42.43  1394.97 

# Now let's run the SATs
We have to run each site at the 50%-tile pwv to get the right psats, then use those for all pwvs.

In [24]:
data = {}

yamlfiles = ['SAT_LFMF_20210714.yaml','SAT_HF_20210714.yaml','CHSAT_LFMF_20210714.yaml','CHSAT_HF_20210714.yaml']


#read fixed PSATs from .toml file
psat_dict = toml.load("FixedPsats.toml")

pwvs = {}
pwvs['Pole']=[321.0, 448.0]
pwvs['Atacama']=[993.0, 1871.0]

for yamlfile in yamlfiles:
    print(yamlfile)
    dd = yaml.safe_load(open(yamlfile))
    
    # Change elevation to 50 degrees
    dd['instrument']['elevation']=50.0

    version_date = str(dd['version']['date'])
    version_telescope = str(dd['version']['name'])
    print('{0:8s}  Popt(pW) Psat(pW)   NET    NET_wafer'.format(version_telescope))
    del dd['version']

    # location of atmosphere file
    dd['sim_config']['config_dir'] = '../../bolo-calc/config'
    
    ch_names = list(dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'].keys())
    
    #---------
    #convert to Watts
    for channel in ch_names:
        psat_dict[yamlfile.partition('.')[0]][channel] *= 1e-12

    #set psat factor to None & delete psat entry from channel default
    dd['instrument']['channel_default']['psat_factor'] = None
    del dd['instrument']['channel_default']['psat']

    #assign SI psat values at the channel level
    for channel in ch_names:
        dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'][channel]['psat'] = psat_dict[yamlfile.partition('.')[0]][channel]
    #---------
    
    site = dd['instrument']['site']
    for pwv in pwvs[site]:
        dd['instrument']['pwv']=pwv
        print('Site=',dd['instrument']['site'])
        print('Elevation = ',dd['instrument']['elevation'])
        print('pwv=',dd['instrument']['pwv'])
        top = Top(**dd)
        top.run()
        
        for chan in top.instrument.tables.keys():
            if chan.split('_')[0] == 'cam' :
                chname = chan.split('_')[2] + '_' + chan.split('_')[3]
                #
                # Grab things we're going to print and give them short names and store them in the dictionary.
                popt = top.instrument.tables[chan]['opt_power'][0]
                psat = top.instrument.tables[chan]['P_sat'][0]
                net = top.instrument.tables[chan]['NET'][0]
                netwafer = top.instrument.tables[chan]['NET_arr'][0]
                #
                print('{0:5s} : {1:8.2f} {2:8.2f} {3:8.2f} {4:8.2f}'.format(chname,popt,psat,net,netwafer) )
        print('')
    print('--------------------')
    

SAT_LFMF_20210714.yaml
SAT_LFMF  Popt(pW) Psat(pW)   NET    NET_wafer
Site= Pole
Elevation =  50.0
pwv= 321.0
LF_1  :     0.57     1.43   175.98    40.16
LF_2  :     2.57     6.42   217.62    49.66
MF1_1 :     3.06     7.86   312.69    20.39
MF1_2 :     5.50    14.25   330.11    21.52
MF2_1 :     3.22     7.86   277.10    18.07
MF2_2 :     5.90    14.25   353.93    23.08

Site= Pole
Elevation =  50.0
pwv= 448.0
LF_1  :     0.58     1.43   177.10    40.42
LF_2  :     2.57     6.42   217.99    49.75
MF1_1 :     3.08     7.86   314.42    20.50
MF1_2 :     5.63    14.25   337.04    21.98
MF2_1 :     3.25     7.86   279.24    18.21
MF2_2 :     6.10    14.25   364.50    23.77

--------------------
SAT_HF_20210714.yaml
SAT_HF    Popt(pW) Psat(pW)   NET    NET_wafer
Site= Pole
Elevation =  50.0
pwv= 321.0
HF_1  :    13.21    33.03   726.90    26.54
HF_2  :    16.25    40.62  1747.15    63.78

Site= Pole
Elevation =  50.0
pwv= 448.0
HF_1  :    13.79    33.03   756.23    27.61
HF_2  :    17.13  